In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader




In [3]:
import torch
from sklearn.metrics import accuracy_score
import numpy as np
import time
import psutil
from pathlib import Path

def compute_metrics_base(model, x_test, y_test, model_path):
    """
    Compute the accuracy of the PyTorch model.

    :param model: PyTorch model.
    :param x_test: Test dataset features (as a PyTorch Tensor).
    :param y_test: Test dataset labels (as a NumPy array).
    :param model_dir: Directory where the PyTorch model files are stored.
    :return: None
    """

    model.eval()
    with torch.no_grad():
        # Get the model's predictions
        outputs = model(x_test)
        _, predicted_labels = torch.max(outputs, 1)

        # Convert y_test to tensor if it's not already
        true_labels = torch.tensor(y_test) if not isinstance(y_test, torch.Tensor) else y_test
        true_labels = true_labels.squeeze()  # Remove unnecessary dimensions

    model_file = Path(model_path)

    # Size in bytes
    model_size_bytes = model_file.stat().st_size

    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

    # Compute accuracy
    accuracy = accuracy_score(true_labels.numpy(), predicted_labels.numpy())
    print(f'Accuracy on the test set: {accuracy:.2%}')
def measure_cpu_utilization_and_run(func, *args, **kwargs):
    """
    Measure CPU utilization while running a function.

    Parameters:
        func (function): The function to be executed.
        *args: Arguments to be passed to func.
        **kwargs: Keyword arguments to be passed to func.

    Returns:
        float: CPU utilization percentage during the execution of func.
        float: The elapsed time during the execution of func.
        any: The result of func execution.
    """
    
    # Measure CPU utilization before execution
    cpu_percent_before = psutil.cpu_percent(interval=None)

    # Record the start time
    start_time = time.time()

    # Execute the function and store its result
    result = func(*args, **kwargs)

    # Record the end time
    end_time = time.time()

    # Measure CPU utilization after execution
    cpu_percent_after = psutil.cpu_percent(interval=None)

    # Calculate elapsed time and average CPU utilization
    elapsed_time = end_time - start_time
    average_cpu_utilization = (cpu_percent_before + cpu_percent_after) / 2

    return average_cpu_utilization, elapsed_time, result



In [4]:
# Load Data
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y_df = pd.read_csv('../Data/WISDM_y.csv')
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [5]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)
print(f'X_train shape:{X_train.shape}, X_test shape:{X_test.shape}, y_train shape:{y_train.shape}, y_test shape:{y_test.shape}')

X_train shape:(109820, 16, 3), X_test shape:(27455, 16, 3), y_train shape:(109820,), y_test shape:(27455,)


In [6]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)  # Assuming y_train is already encoded as class indexes
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)


In [7]:
# Create Dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


In [8]:
# Define the Transformer model
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerEncoderBlock(nn.Module):
    def __init__(self, input_dim, head_size, n_heads, ff_dim, dropout=0.0):
        super(TransformerEncoderBlock, self).__init__()
        self.norm1 = nn.LayerNorm(input_dim)
        self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=n_heads, dropout=dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(input_dim)
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=ff_dim, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=ff_dim, out_channels=input_dim, kernel_size=1)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        # LayerNorm and Multi-head Attention
        x = self.norm1(src)
        x, _ = self.attention(x, x, x)
        x = self.dropout1(x)
        x = x + src  # skip connection

        # Feed Forward
        x = self.norm2(x)
        x = x.permute(1, 2, 0)  # Conv1D expects (batch_size, channels, length)
        x = F.relu(self.conv1(x))
        x = self.dropout2(x)
        x = self.conv2(x)
        x = x.permute(2, 0, 1)  # back to (length, batch_size, channels)
        x = x + src  # skip connection
        return x

class TimeSeriesTransformer(nn.Module):
    def __init__(self, sequence_length, num_features, head_size, n_heads, ff_dim, n_trans_blocks, mlp_units, drop=0.0, mlp_drop=0.0):
        super(TimeSeriesTransformer, self).__init__()
        self.encoders = nn.ModuleList([TransformerEncoderBlock(num_features, head_size, n_heads, ff_dim, drop) for _ in range(n_trans_blocks)])
        self.global_avg_pooling = nn.AdaptiveAvgPool1d(1)
        mlp_layers = []
        current_dim = num_features
        for dim in mlp_units:
            mlp_layers.append(nn.Linear(current_dim, dim))
            mlp_layers.append(nn.ReLU())
            mlp_layers.append(nn.Dropout(mlp_drop))
            current_dim = dim  # Set input dim for the next layer
        self.mlp = nn.Sequential(*mlp_layers)
        self.final_layer = nn.Linear(mlp_units[-1], 6)

    def forward(self, src):
        src = src.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, features)
        for encoder in self.encoders:
            src = encoder(src)

        # Global average pooling
        src = src.permute(1, 2, 0)  # pooling expects (batch_size, channels, length)
        src = self.global_avg_pooling(src)
        src = torch.flatten(src, 1)  # Flatten the output for the MLP

        # MLP
        src = self.mlp(src)
        output = self.final_layer(src)
        return output

# Input parameters for your data
sequence_length = 16  # The length of the time series sequences in your data
num_features = 3     # The number of features in each time step of your data sequence

# Instantiate the model
# Instantiate the model with an adjusted number of heads and head size
# The head size must be a multiple of num_features.
model = TimeSeriesTransformer(
    sequence_length=16, 
    num_features=3, 
    head_size=3,  # Each head will now have an embed size of 1 (3 / 3)
    n_heads=1,  # Only one head since our embed_dim is 3
    ff_dim=64, 
    n_trans_blocks=4, 
    mlp_units=[128, 64], 
    drop=0.1, 
    mlp_drop=0.1
)




In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


TimeSeriesTransformer(
  (encoders): ModuleList(
    (0-3): 4 x TransformerEncoderBlock(
      (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
      (attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)
      )
      (dropout1): Dropout(p=0.1, inplace=False)
      (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 3, kernel_size=(1,), stride=(1,))
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (global_avg_pooling): AdaptiveAvgPool1d(output_size=1)
  (mlp): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
  )
  (final_layer): Linear(in_features=64, out_features=6, bias=True)
)

In [10]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], '
                  f'Loss: {running_loss / 100:.4f}, Accuracy: {100 * correct / total:.2f}%')
            running_loss = 0.0
            correct = 0
            total = 0

# After training, you may want to save your model
# torch.save(model.state_dict(), 'model.pth')

print('Finished Training')


Epoch [1/10], Step [100/3432], Loss: 1.3419, Accuracy: 48.47%
Epoch [1/10], Step [200/3432], Loss: 1.1859, Accuracy: 57.31%
Epoch [1/10], Step [300/3432], Loss: 1.0795, Accuracy: 63.03%
Epoch [1/10], Step [400/3432], Loss: 0.9647, Accuracy: 67.78%
Epoch [1/10], Step [500/3432], Loss: 0.8956, Accuracy: 70.47%
Epoch [1/10], Step [600/3432], Loss: 0.8596, Accuracy: 72.94%
Epoch [1/10], Step [700/3432], Loss: 0.8099, Accuracy: 72.69%
Epoch [1/10], Step [800/3432], Loss: 0.7955, Accuracy: 73.19%
Epoch [1/10], Step [900/3432], Loss: 0.7544, Accuracy: 74.09%
Epoch [1/10], Step [1000/3432], Loss: 0.7499, Accuracy: 74.84%
Epoch [1/10], Step [1100/3432], Loss: 0.7513, Accuracy: 74.44%
Epoch [1/10], Step [1200/3432], Loss: 0.7958, Accuracy: 72.09%
Epoch [1/10], Step [1300/3432], Loss: 0.7440, Accuracy: 73.91%
Epoch [1/10], Step [1400/3432], Loss: 0.7140, Accuracy: 75.94%
Epoch [1/10], Step [1500/3432], Loss: 0.7192, Accuracy: 75.22%
Epoch [1/10], Step [1600/3432], Loss: 0.7131, Accuracy: 75.53%
E

Epoch [4/10], Step [3000/3432], Loss: 0.6158, Accuracy: 78.12%
Epoch [4/10], Step [3100/3432], Loss: 0.6059, Accuracy: 78.94%
Epoch [4/10], Step [3200/3432], Loss: 0.6153, Accuracy: 78.56%
Epoch [4/10], Step [3300/3432], Loss: 0.5908, Accuracy: 78.84%
Epoch [4/10], Step [3400/3432], Loss: 0.6187, Accuracy: 78.25%
Epoch [5/10], Step [100/3432], Loss: 0.6323, Accuracy: 78.16%
Epoch [5/10], Step [200/3432], Loss: 0.6301, Accuracy: 78.50%
Epoch [5/10], Step [300/3432], Loss: 0.6310, Accuracy: 77.50%
Epoch [5/10], Step [400/3432], Loss: 0.5925, Accuracy: 79.06%
Epoch [5/10], Step [500/3432], Loss: 0.5740, Accuracy: 80.28%
Epoch [5/10], Step [600/3432], Loss: 0.6154, Accuracy: 78.84%
Epoch [5/10], Step [700/3432], Loss: 0.5966, Accuracy: 79.38%
Epoch [5/10], Step [800/3432], Loss: 0.5926, Accuracy: 78.72%
Epoch [5/10], Step [900/3432], Loss: 0.6111, Accuracy: 79.38%
Epoch [5/10], Step [1000/3432], Loss: 0.5799, Accuracy: 79.47%
Epoch [5/10], Step [1100/3432], Loss: 0.5950, Accuracy: 79.06%
E

Epoch [8/10], Step [2500/3432], Loss: 0.5771, Accuracy: 80.03%
Epoch [8/10], Step [2600/3432], Loss: 0.5691, Accuracy: 79.91%
Epoch [8/10], Step [2700/3432], Loss: 0.5755, Accuracy: 80.84%
Epoch [8/10], Step [2800/3432], Loss: 0.5608, Accuracy: 80.38%
Epoch [8/10], Step [2900/3432], Loss: 0.5671, Accuracy: 80.53%
Epoch [8/10], Step [3000/3432], Loss: 0.5634, Accuracy: 79.81%
Epoch [8/10], Step [3100/3432], Loss: 0.5831, Accuracy: 79.88%
Epoch [8/10], Step [3200/3432], Loss: 0.5467, Accuracy: 80.75%
Epoch [8/10], Step [3300/3432], Loss: 0.5771, Accuracy: 80.03%
Epoch [8/10], Step [3400/3432], Loss: 0.5851, Accuracy: 79.28%
Epoch [9/10], Step [100/3432], Loss: 0.5721, Accuracy: 80.38%
Epoch [9/10], Step [200/3432], Loss: 0.5620, Accuracy: 79.84%
Epoch [9/10], Step [300/3432], Loss: 0.5900, Accuracy: 79.41%
Epoch [9/10], Step [400/3432], Loss: 0.5784, Accuracy: 80.25%
Epoch [9/10], Step [500/3432], Loss: 0.5652, Accuracy: 80.34%
Epoch [9/10], Step [600/3432], Loss: 0.5818, Accuracy: 79.94

In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Testing loop
model.eval()  # Set the model to evaluation mode
all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        # Collect all predictions and labels to compute overall metrics
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

# Convert collected predictions and labels to arrays
all_predictions = np.array(all_predictions)
all_targets = np.array(all_targets)

# Calculate metrics
accuracy = accuracy_score(all_targets, all_predictions)
precision, recall, f1_score, support = precision_recall_fscore_support(all_targets, all_predictions, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1_score:.4f}')


Accuracy: 0.8076
Precision: 0.7903
Recall: 0.8076
F1 Score: 0.7821


In [12]:
import torch
import os

# Assume 'model' is the instance of TimeSeriesTransformer you have already defined and trained
model_path = "/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/transformer_base.pth"
torch.save(model.state_dict(), model_path)

# Get the size of the saved model file
model_size = os.path.getsize(model_path)
print(f"The model size is {model_size/1024:.2f} KB")


The model size is 61.19 KB


In [13]:
cpu_usage, inference_time, _ = measure_cpu_utilization_and_run(compute_metrics_base, model, X_test_tensor, y_test_tensor, model_path)

print(f'CPU usage during inference: {cpu_usage:.2f}%')
print(f'Inference time: {inference_time:.4f} seconds')


Size of the model: 61.19 KB
Accuracy on the test set: 80.76%
CPU usage during inference: 39.80%
Inference time: 1.5054 seconds


In [18]:
import torch.quantization
torch.backends.quantized.engine = 'qnnpack'

# Load the saved model's state dict
model.load_state_dict(torch.load(model_path))

# Make sure the model is in evaluation mode before quantization
model.eval()

# Perform dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # specify which layer types to quantize
    dtype=torch.qint8  # the target data type for quantized weights
)

# Save the quantized model
quantized_model_path = "/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/transformer_quantized.pth"
torch.save(quantized_model.state_dict(), quantized_model_path)

# Now you can use quantized_model for inference
print("Quantization complete and model saved.")


Quantization complete and model saved.


In [19]:
# Check the size of the quantized model
quantized_model_size = os.path.getsize(quantized_model_path)
print(f"Size of quantized model: {quantized_model_size/1024:0.2f} KB")


Size of quantized model: 37.35 KB


In [20]:
# Testing loop
quantized_model.eval()  # Set the model to evaluation mode
all_predictions = []
all_targets = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = quantized_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        # Collect all predictions and labels to compute overall metrics
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

# Convert collected predictions and labels to arrays
all_predictions = np.array(all_predictions)
all_targets = np.array(all_targets)

# Calculate metrics
accuracy = accuracy_score(all_targets, all_predictions)
precision, recall, f1_score, support = precision_recall_fscore_support(all_targets, all_predictions, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1_score:.4f}')


[W qlinear_dynamic.cpp:247] Warning: Currently, qnnpack incorrectly ignores reduce_range when it is set to true; this may change in a future release. (function operator())


Accuracy: 0.8080
Precision: 0.7909
Recall: 0.8080
F1 Score: 0.7822


In [21]:
cpu_usage, inference_time, _ = measure_cpu_utilization_and_run(compute_metrics_base, quantized_model, X_test_tensor, y_test_tensor, quantized_model_path)

print(f'CPU usage during inference: {cpu_usage:.2f}%')
print(f'Inference time: {inference_time:.4f} seconds')


Size of the model: 37.35 KB
Accuracy on the test set: 80.66%
CPU usage during inference: 35.75%
Inference time: 1.4863 seconds


### Static Quantization

In [22]:
# Define the Transformer model
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerEncoderBlock(nn.Module):
    def __init__(self, input_dim, head_size, n_heads, ff_dim, dropout=0.0):
        super(TransformerEncoderBlock, self).__init__()
        self.norm1 = nn.LayerNorm(input_dim)
        self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=n_heads, dropout=dropout)
        self.dropout1 = nn.Dropout(dropout)
        self.norm2 = nn.LayerNorm(input_dim)
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=ff_dim, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=ff_dim, out_channels=input_dim, kernel_size=1)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        # LayerNorm and Multi-head Attention
        x = self.norm1(src)
        x, _ = self.attention(x, x, x)
        x = self.dropout1(x)
        x = x + src  # skip connection

        # Feed Forward
        x = self.norm2(x)
        x = x.permute(1, 2, 0)  # Conv1D expects (batch_size, channels, length)
        x = F.relu(self.conv1(x))
        x = self.dropout2(x)
        x = self.conv2(x)
        x = x.permute(2, 0, 1)  # back to (length, batch_size, channels)
        x = x + src  # skip connection
        return x

class TimeSeriesTransformer(nn.Module):
    def __init__(self, sequence_length, num_features, head_size, n_heads, ff_dim, n_trans_blocks, mlp_units, drop=0.0, mlp_drop=0.0):
        super(TimeSeriesTransformer, self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.encoders = nn.ModuleList([TransformerEncoderBlock(num_features, head_size, n_heads, ff_dim, drop) for _ in range(n_trans_blocks)])
        self.global_avg_pooling = nn.AdaptiveAvgPool1d(1)
        mlp_layers = []
        current_dim = num_features
        for dim in mlp_units:
            mlp_layers.append(nn.Linear(current_dim, dim))
            mlp_layers.append(nn.ReLU())
            mlp_layers.append(nn.Dropout(mlp_drop))
            current_dim = dim  # Set input dim for the next layer
        self.mlp = nn.Sequential(*mlp_layers)
        self.final_layer = nn.Linear(mlp_units[-1], 6)
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, src):
        src = self.quant(src)
        src = src.permute(1, 0, 2)  # Transformer expects (seq_len, batch_size, features)
        for encoder in self.encoders:
            src = encoder(src)

        # Global average pooling
        src = src.permute(1, 2, 0)  # pooling expects (batch_size, channels, length)
        src = self.global_avg_pooling(src)
        src = torch.flatten(src, 1)  # Flatten the output for the MLP

        # MLP
        src = self.mlp(src)
        src = self.final_layer(src)
        src = self.dequant(src)
        return src

# Input parameters for your data
sequence_length = 16  # The length of the time series sequences in your data
num_features = 3     # The number of features in each time step of your data sequence

# Instantiate the model
# Instantiate the model with an adjusted number of heads and head size
# The head size must be a multiple of num_features.
model_qat = TimeSeriesTransformer(
    sequence_length=16, 
    num_features=3, 
    head_size=3,  # Each head will now have an embed size of 1 (3 / 3)
    n_heads=1,  # Only one head since our embed_dim is 3
    ff_dim=64, 
    n_trans_blocks=4, 
    mlp_units=[128, 64], 
    drop=0.1, 
    mlp_drop=0.1
)

In [23]:
model_qat.qconfig = torch.quantization.get_default_qat_qconfig('qnnpack')
model_qat.train()
model_prepared = torch.quantization.prepare_qat(model_qat, inplace=True)


AssertionError:  qat.Linear.from_float only works for Linear

In [31]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune

# Instantiate the model
pruned_model = TimeSeriesTransformer(
    sequence_length=16, 
    num_features=3, 
    head_size=3,
    n_heads=1, 
    ff_dim=64, 
    n_trans_blocks=4, 
    mlp_units=[128, 64], 
    drop=0.1, 
    mlp_drop=0.1
)

# Apply pruning to convolutional layers in each TransformerEncoderBlock
for encoder_block in pruned_model.encoders:
    prune.l1_unstructured(encoder_block.conv1, 'weight', amount=0.1)
    prune.l1_unstructured(encoder_block.conv2, 'weight', amount=0.1)

# Apply pruning to linear layers in the MLP
for layer in pruned_model.mlp:
    if isinstance(layer, nn.Linear):
        prune.l1_unstructured(layer, 'weight', amount=0.1)

# To make the pruning permanent
for encoder_block in pruned_model.encoders:
    prune.remove(encoder_block.conv1, 'weight')
    prune.remove(encoder_block.conv2, 'weight')

for layer in pruned_model.mlp:
    if isinstance(layer, nn.Linear):
        prune.remove(layer, 'weight')


In [32]:
pruned_model_path = "/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/Transformer_pruned.pth"
torch.save(pruned_model.state_dict(), pruned_model_path)


In [33]:
cpu_usage, inference_time, _ = measure_cpu_utilization_and_run(compute_metrics_base, pruned_model, X_test_tensor, y_test_tensor, pruned_model_path)

print(f'CPU usage during inference: {cpu_usage:.2f}%')
print(f'Inference time: {inference_time:.4f} seconds')


Size of the model: 61.36 KB
Accuracy on the test set: 22.25%
CPU usage during inference: 34.55%
Inference time: 1.4775 seconds
